# Module 1: Introduction to Machine Learning and Data Processing for Food Sciences

Welcome to the first module of this series! In this module you will first get an overview on what Machine Learning is and how it can be used in Food Science. Then we will move on to a real-world dataset from the [Swiss Food composition database](https://naehrwertdaten.ch/en/) to learn what data pre-processing steps are needed before applying Machine Learning methods on our data. We will be using this processed dataset in the next two modules on supervised and unsupervised Machine Learning.

**Module overview**
1. [Introduction to Machine Learning (ML)](#intro)
2. [ML in Food Science](#food)
3. [Basic data processing for ML](#proc)

**Dataset**       
Throughout this and future modules we will be working with a dataset from the [Swiss Food composition database](https://naehrwertdaten.ch/en/). The dataset contains information on the composition of different foods that are available in Switzerland and is maintained by the Federal Food Safety and Veterinary Office. In this tutorial you will be given a pre-cleaned version of this dataset. If you are interested in the cleaning performed on the original dataset, feel free to look at the following script: `scripts/Clean_FoodCompositionData.ipynb`.

<a id='intro'></a>
## 1. Introduction to Machine Learning (ML)

Machine Learning (ML) has become increasingly prevalent in our daily lives. Text writing programs use ML to autocorrect your texts, search engines use ML to improve your search results, and spam in your email inbox is detected by ML. Although ML has gained significant popularity and widespread use in recent years, it is not a new concept. The term was first coined by Arthur Samuel in 1959. He defined Machine Learning as "the field of study that gives computers the ability to learn without being explicitly programmed". To rephrase this in simpler terms, one can say that **Machine Learning is the field of study that gives computers the ability to learn from data**. 

Data is essential for any ML method. Depending on the type of data we have, we differentiate between **supervised and unsupervised ML**. If the data we have is labelled and we know the groups within, we can use supervised ML methods. Conversely, if our data is not labelled or we decide to ignore any known labels, we can employ unsupervised ML techniques to identify patterns within. You will learn more about both two types of ML in future modules.

There are three distinct steps in **building a ML model** in practice (see [Figure 1](#ml-model)):        
**1)** processing the data     
**2)** training the ML model    
**3)** evaluating the trained model      

The quality of the ML model depends a lot on the quality of the data available to train it. Therefore, a lot of time is spent in practice on processing the raw data (**1) PROCESS**). You will learn more about the usual data processing steps in the [respective section later on](#proc). Once the data is processed, it is split into two parts: the train and test set. The train set is used to train the ML model (**2) TRAIN**) whereas the test set is used to evaluate the model performance (**3) EVALUATE**).        
Once we have found a ML model that performs well on our test set, we can use it to:           
* gain <span style="color:darkred">insights into the processes governing our data</span>
* perform <span style="color:darkgreen">predictions on new data</span>.


<center>
<a id="ml-model"></a>
<img src="images/part1_intro/ml_pipeline.jpg" alt="Machine Learning Model" width="75%">
<center><figcaption><em>Figure 1: Three steps in building a ML model.</em></figcaption></center>
</center>

**💡 CHECKPOINT:**
 - You have trained a supervised ML model to accurately predict the expiration date of a food product given its chemical composition. What could you do with the trained model if someone gave you an unlabelled set of foods and their compositions?

<a id='food'></a>
## 2. ML in Food Science

<span style="color:red">TODO: Input from experts needed.</span> Draft text from Doriela can be found in `notes/m1_ml_in_food_science.txt`.

<a id='proc'></a>
## 3. Basic data processing for ML

Since ML models are built from data, the performance of the trained models depends on the quality of the data fed into the model. In practice, the data we get from collaborators or the internet might not be standardized. Also, the data might contain erroneous or missing values. Hence, it is crucial to explore and process the data before we use it for training ML models. 

In this section we will introduce some frequently used data pre-processing methods on the Swiss food composition dataset. The pre-processing methods we will cover include:
* [3.2 Data quality control](#quality)
* [3.3 Train-test split](#train-test-split)
* [3.4 Standardization](#stand)

Before diving into these processing methods, we will explore what our dataset is composed of in section [3.1 Anatomy of our data](#anatomy).

To explore and process our dataset we will make use of different Python modules that are imported in the cell below.


In [38]:
# Used to read the dataset
import os

# Pandas is used to read and interact with our data across all processing steps
# we import it with an alias "pd"
import pandas as pd

# The below packages are additionally needed for sections 3.2 - 3.4
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Packages needed for visualizations
import matplotlib.pyplot as plt
import seaborn as sns

When the above cell finishes running successfully, you will see `[1]`, in the bottom left corner of the box. This means that the packages are loaded and ready to be used.

### 3.1 Anatomy of our data <a id='anatomy'></a> 

By data we refer to the collection of samples and features. Each **sample** represents a row in our table and is denoted with a unique ID. For each sample, we get a set of **features** that were measured. Features represent certain characteristics of our samples. 

Let us illustrate samples and features structure on our Swiss food composition dataset. To do this, we first have to **load the data**. The dataset is stored in a `.csv` (comma separated values) file and we will save it in a **pandas dataframe**.

In [39]:
filepath = os.path.join("data", "swiss_food_composition_proc.csv")
dataset = pd.read_csv(filepath, index_col=0)

In order to find how many samples and features we have in our dataset, we can use the `shape` attribute of the `dataset` dataframe. 

In [40]:
dataset.shape

(1084, 41)

Now, we know that there are 1092 samples (rows) and 41 features (columns) in our dataset.

To see how the first 5 rows of our dataset look we can use the `head()` method.

In [41]:
dataset.head()

,name,category,energy_kcal,fat_g,fatty_acids_sat_g,fatty_acids_monounsat_g,fatty_acids_polyunsat_g,cholesterol_mg,carbohydrates_g,sugars_g,...,potassium_mg,sodium_mg,chloride_mg,calcium_mg,magnesium_mg,phosphorus_mg,iron_mg,iodide_µg,zinc_mg,split
ID,,,,,,,,,,,,,,,,,,,,,
2,Almond,fruits,2.383884,2.367211,0.007568,3.747869,1.491113,-0.536758,-0.392082,0.004006,...,1.588591,-0.159777,-0.148160,1.295299,4.040213,2.312243,0.749351,-0.106408,1.315695,train
3,"Almond, dry roasted, salted",nuts,2.458504,2.390231,0.007568,3.980279,1.745907,-0.536758,-0.285661,-0.117252,...,1.488645,-0.083438,0.108003,1.295299,4.816868,2.059374,0.906205,-0.088162,1.315695,train
4,"Almond, roasted, salted",nuts,2.527384,2.545613,0.020843,4.212688,1.825530,-0.536758,-0.419844,-0.138650,...,1.355384,-0.050088,0.108003,1.098111,4.622704,2.059374,0.749351,-0.088162,1.185016,test
5,"Amaranth, seed, cooked (without addition of fa...",cereals,-0.514820,-0.510234,-0.483593,-0.476516,-0.196900,-0.536758,0.084497,-0.431096,...,-0.377014,-0.159511,-0.149051,-0.098161,1.011260,0.226077,0.631711,-0.101432,-0.056440,test
6,"Amaranth, seed, raw",cereals,0.960362,-0.228244,-0.337572,-0.312462,0.121593,-0.536758,1.875138,-0.345502,...,0.722392,-0.158810,-0.132568,0.572278,4.234377,2.628328,2.984515,-0.087333,1.577054,train


To see how the last 5 rows of our dataset look we can use the `tail()` method.

In [42]:
dataset.tail()

,name,category,energy_kcal,fat_g,fatty_acids_sat_g,fatty_acids_monounsat_g,fatty_acids_polyunsat_g,cholesterol_mg,carbohydrates_g,sugars_g,...,potassium_mg,sodium_mg,chloride_mg,calcium_mg,magnesium_mg,phosphorus_mg,iron_mg,iodide_µg,zinc_mg,split
ID,,,,,,,,,,,,,,,,,,,,,
1087,"Zucchini piccata, prepared",other,-0.48612,-0.170695,-0.244650,0.001974,-0.165051,0.401320,-0.447606,-0.338369,...,-0.177122,-0.129128,-0.123657,0.020152,-0.153721,-0.298625,-0.191770,-0.016840,-0.383139,train
1088,"Zucchini slices, breaded, prepared",other,-0.46890,-0.314568,-0.443769,-0.080052,-0.180975,0.098249,-0.128344,-0.316971,...,-0.177122,-0.130462,-0.121430,-0.295348,-0.153721,-0.500920,-0.152557,-0.047525,-0.513818,train
1089,"Zucchini, raw",vegetables,-1.08882,-0.619577,-0.536691,-0.544872,-0.308373,-0.536758,-0.660447,-0.331236,...,-0.110492,-0.159144,-0.151724,-0.354505,-0.173138,-0.715858,-0.230984,-0.088992,-0.709838,train
1090,"Zucchini, steamed (without addition of salt)",vegetables,-1.08308,-0.619577,-0.536691,-0.544872,-0.308373,-0.536758,-0.651193,-0.316971,...,-0.143807,-0.159177,-0.151279,-0.341359,-0.153721,-0.703215,-0.270197,-0.087333,-0.644498,train
1091,"Zucchini, stewed (without addition of fat and ...",vegetables,-1.06586,-0.619577,-0.536691,-0.544872,-0.308373,-0.536758,-0.641939,-0.302705,...,-0.077176,-0.159010,-0.150388,-0.321640,-0.134305,-0.665285,-0.191770,-0.084016,-0.644498,train


We can see in the above data excerpts that each sample is linked with a unique row `ID` ranging from `1` to `1091`. 

### 3.2 Data Quality Control <a id='quality'></a>

The first step in the data pre-processing phase is to assess the quality of our data. There are different ways to inspect this and here we will explore some of them.

#### Find number of missing values

Let us start by getting some information about what kind of data is contained in the dataset. For this we will use the `info()` method of the `dataset`. Besides listing the data types of each column, it shows how many non-empy (`non-null`) values are there for each feature.

In [43]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1084 entries, 2 to 1091
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   name                       1084 non-null   object 
 1   category                   1084 non-null   object 
 2   energy_kcal                1084 non-null   float64
 3   fat_g                      1084 non-null   float64
 4   fatty_acids_sat_g          1084 non-null   float64
 5   fatty_acids_monounsat_g    1084 non-null   float64
 6   fatty_acids_polyunsat_g    1084 non-null   float64
 7   cholesterol_mg             1084 non-null   float64
 8   carbohydrates_g            1084 non-null   float64
 9   sugars_g                   1084 non-null   float64
 10  starch_g                   1084 non-null   float64
 11  fibres_g                   1084 non-null   float64
 12  protein_g                  1084 non-null   float64
 13  salt_g                     1084 non-null   float

Most of the columns hold decimal values, which in Python correspond to the **float64** data types.
The first two columns, `Name` and `Category` are `strings` (sequences of characters) and `pandas` recognizes them as objects.

The missing values in the dataset are recorded as `NaN`s. Here is an example: 

In [44]:
dataset.iloc[0:1, :]

,name,category,energy_kcal,fat_g,fatty_acids_sat_g,fatty_acids_monounsat_g,fatty_acids_polyunsat_g,cholesterol_mg,carbohydrates_g,sugars_g,...,potassium_mg,sodium_mg,chloride_mg,calcium_mg,magnesium_mg,phosphorus_mg,iron_mg,iodide_µg,zinc_mg,split
ID,,,,,,,,,,,,,,,,,,,,,
2,Almond,fruits,2.383884,2.367211,0.007568,3.747869,1.491113,-0.536758,-0.392082,0.004006,...,1.588591,-0.159777,-0.14816,1.295299,4.040213,2.312243,0.749351,-0.106408,1.315695,train


For `Agar Agar` there is no value for `fatty_acids_sat_g`, `fatty_acids_monounsat_g`, `fatty_acids_polyunsat_g`, etc.

#### Removing features with a lot of missing values

The features (columns) that have a lot of missing values will not be useful for the models, so we can remove them. Determining what *a lot* means depends on the context and task at hand. In our case, we will remove the features that have more than 20% of the values missing. For this we will first find the actual minimum number of rows that have to contain a value per feature. This will be the `feature_threshold`. 

Since we will remove features that have more than 20% of the data missing, we will keep those that have at least 80%. We have 1092 samples in total, this means that we will keep features that have at least $$\left\lceil 0.8*1092=873.6\right\rceil = 874$$ non-missing values. 

First, we will print the number of `NaN` values per column.

In [45]:
nan_total_col = dataset.isna().sum(axis=0)
nan_total_col

name                         0
category                     0
energy_kcal                  0
fat_g                        0
fatty_acids_sat_g            0
fatty_acids_monounsat_g      0
fatty_acids_polyunsat_g      0
cholesterol_mg               0
carbohydrates_g              0
sugars_g                     0
starch_g                     0
fibres_g                     0
protein_g                    0
salt_g                       0
alcohol_g                    0
water_g                      0
vit_A_activity_re_µg         0
vit_A_activity_rae_µg        0
retinol_µg                   0
beta_carotene_activity_µg    0
beta_carotene_µg             0
vit_B1_mg                    0
vit_B2_mg                    0
vit_B6_mg                    0
vit_B12_µg                   0
niacin_mg                    0
folate_µg                    0
panthotenic_acid_mg          0
vit_c_mg                     0
vit_d_µg                     0
vit_e_activity_mg            0
potassium_mg                 0
sodium_m

From the list above, we will remove all columns that have more than 1092-874 = 218 missing values. Only the `selenium_µg` has more than 218 missing values.

The above code cell does the following:

It checks each value of the dataset and marks it with 1 if it is a `nan` and with a 0 otherwise. Then, it computes the sum of each column using the `sum()` method with the `axis=0`. In this way we find the number of 1s per column, which means the number of values that are `nan`s per column. You can find a visual explanation on [Figure 2](#boolean-mask-illustration) below.

Now we can drop the columns that have more than 20% of the values missing:

In [46]:
percentage_present = 0.8
feature_threshold = math.ceil(percentage_present*dataset.shape[0]) # 20% of values missing
feature_threshold

868

In [47]:
columns_before = dataset.columns
dataset = dataset.dropna(axis=1, thresh=feature_threshold)
columns_after = dataset.columns
columns_after

Index(['name', 'category', 'energy_kcal', 'fat_g', 'fatty_acids_sat_g',
       'fatty_acids_monounsat_g', 'fatty_acids_polyunsat_g', 'cholesterol_mg',
       'carbohydrates_g', 'sugars_g', 'starch_g', 'fibres_g', 'protein_g',
       'salt_g', 'alcohol_g', 'water_g', 'vit_A_activity_re_µg',
       'vit_A_activity_rae_µg', 'retinol_µg', 'beta_carotene_activity_µg',
       'beta_carotene_µg', 'vit_B1_mg', 'vit_B2_mg', 'vit_B6_mg', 'vit_B12_µg',
       'niacin_mg', 'folate_µg', 'panthotenic_acid_mg', 'vit_c_mg', 'vit_d_µg',
       'vit_e_activity_mg', 'potassium_mg', 'sodium_mg', 'chloride_mg',
       'calcium_mg', 'magnesium_mg', 'phosphorus_mg', 'iron_mg', 'iodide_µg',
       'zinc_mg', 'split'],
      dtype='object')

In the first code cell we declare a variable, `feature_threshold = math.ceil(0.8*dataset.shape[0])` to be the threshold according to which we are discarding columns. It gets the number of samples using the first entry of the `dataset.shape` attribute which returns a `tuple`. Then, we save the columns we had to see later what was removed. Afterwards, we apply the `dropna()` method to drop all columns (determined by the `axis=1` argument), that have less than `feature_threshold` non-missing values. We save the columns after the drop as well.

In [48]:
columns_before.difference(columns_after)

Index([], dtype='object')

The `difference` method does set difference on the two lists specified. As you can see, `selenium_µg` is not present anymore in the columns list. The code cell below counts the number of columns. Now we have 40 columns, from 41 that we had before.

In [49]:
len(dataset.columns)

41

#### Removing samples with a lot of missing values

We will remove the samples that have more than  20% of the values missing. This means that we will keep the samples that have more than 80% of the features. With some quick math we find that: $$\left\lceil0.8*41 = 32.8 \right\rceil = 33$$ This means that we will keep the samples that have 33 or more features.

In [50]:
nan_total_rows = dataset.isna().sum(axis=1)
nan_total_rows

ID
2       0
3       0
4       0
5       0
6       0
       ..
1087    0
1088    0
1089    0
1090    0
1091    0
Length: 1084, dtype: int64

The code cell above finds the number of missing values per each sample. This time we sum row-wise by specifying the `axis=1` argument in the `sum()` method. [Figure 2](#boolean-mask-illustration) illustrates how the `isna()` and `sum()` methods work to calculate the number of `NaN`s per each sample and per each feature. Note that we just show an example of the output for the first 3 samples and 8 columns. After creating the boolean masks ('tables' with 0s and 1s), it finds the sum per row or per column, based on the `axis` value we pass as argument to `sum()` and find the number of missing values.

<center>
    <a id="boolean-mask-illustration"></a>
    <img src="images/part1_intro/rows_columns_nans_v3.jpg" alt="Rows and columns boolean mask" width="100%">
    <center><figcaption><em>Figure 2: Compute number of missing values per row and column</em></figcaption></center>
</center>

Now we will remove the rows that have more than 20% of the values missing:

In [51]:
row_threshold = math.ceil(percentage_present*dataset.shape[1]) # 20% of values missing
row_threshold

33

We declare `row_threshold = math.ceil(0.8*dataset.shape[1])` to be the threshold according to which we are discarding rows. It gets the number of features using the second entry of the `dataset.shape` attribute which returns a `tuple`.

In [52]:
rows_before = dataset.index
dataset = dataset.dropna(axis=0, thresh=row_threshold)
rows_after = dataset.index
dataset.shape

(1084, 41)

From 1092 samples, now we have 1084.

In [53]:
rows_before.difference(rows_after)

Int64Index([], dtype='int64', name='ID')

Again we apply the `difference()` method to find the positions of the samples that were dropped.

After removing the features and samples that had a lot of missing values, the dataset has 1084 samples and 40 features. Note that our dataset still has missing values and we will deal with them in the subsequent sections.

#### Outlier detection

Outliers are samples that differ from the majority of the samples in the dataset. Most of the machine learning models are sensitive to outliers since they affect the learning process. Thus it is important to deal with them. **The process of identifying and handling outliers is called "outlier detection".** 

There are multiple ways to detect outliers. They include:
 - **statistical approaches**: calculating mean, standard deviation, quartiles or percentiles of features and identifying data points that fall outside certain ranges or thresholds,
 - **distance-based methods**: e.g using distances like Euclidean distance to find samples that are far away,
 - **machine learning techniques**: using clustering, anomaly detection algorithms, etc.
 
 You can find a more detailed description of outlier detection methods [here](https://www.mygreatlearning.com/blog/what-is-outlier-detection/) and [here](https://towardsdatascience.com/how-to-identify-outliers-in-data-with-python-a9fe40235064).


There are multiple ways to deal with the outliers, too. The easiest of them is to completely remove them. However, we should be careful with this approach since it may result in loss of useful information. Also, if the dataset is small, removing the outliers will make it even smaller. Another approach is to use statistical models or machine learning algorithms that are robust to outliers. More  information on outlier handling techniques can be found [here](https://towardsdatascience.com/dont-throw-away-your-outliers-c37e1ab0ce19).


It depends on the dataset which combination of the above techniques is best suited. Often, researchers test multiple techniques before making a decision.

Our food composition dataset does not contain any outliers. Hence, we will not be performing any outlier detection here. If you are interested in looking into outlier detection methods more closely, you can find them demonstrated with coding examples [here](https://towardsdatascience.com/5-outlier-detection-methods-that-every-data-enthusiast-must-know-f917bf439210).

#### Dealing with missing data

There are several ways to deal with missing data. The easiest and most drastic one is to completely remove the samples that have missing data. We cannot apply this here because we would have to delete more than 70% of the dataset, meaning that we would have too few samples to do any meaningful analysis. Other approaches are possible for imputing the dataset, e.g. replace the missing values with some characteristic values of the features. For example, we can decide to substitute missing values in a column with the mean value for that feature. Likewise we can substitute them with the median or mode values. You can find more imputation techniques in the [sklearn documentation](https://scikit-learn.org/stable/modules/impute.html).

Here we will impute the missing values with the mean of the features, but for each food category. Let us check which food categories do we have in our dataset:

In [54]:
dataset['category'].unique()

array(['fruits', 'nuts', 'cereals', 'sweets', 'other', 'dairy',
       'non_alcoholic_beverages', 'vegetables', 'meat', 'herbs', 'sauce',
       'alcoholic_beverages'], dtype=object)

The above code cell does the following: it first selects all the rows of the `category` column of the dataset and then find all unique values in this column. There are 12 unique food categories in our dataset. In this case, we will find the mean value per food category and per feature, and aftewards, subsitute the missing values with the per category and per feature mean.
Let us look at an example:

In [55]:
dataset.loc[221, ['starch_g', 'category']]

starch_g   -0.524086
category        meat
Name: 221, dtype: object

This sample, that corresponds to the `meat` category has a missing value for the feature `starch_g`.  Using the `loc` indexer we can select the row 221, and for that row we get only the values that correspond to the  `'starch_g'` ,`'category'` columns. This value will be imputed with the mean of all the `starch_g` values for all food samples belonging to the `meat` category.
To do so we will write a function:

In [56]:
def impute_missing_values(dataset:pd.DataFrame, 
                          category_col:str, 
                          feature_col_names:list)->pd.DataFrame:
    modified_dataset = dataset.copy()
    for feature in feature_col_names:
            modified_dataset[feature] = modified_dataset.groupby(category_col)[feature]\
                                                        .transform(lambda x: x.fillna(x.mean()))
    return modified_dataset

This function takes as arguments the dataset that we are imputing, the name of the column that specifies the category according to which we are grouping the food samples and the feature columns (all the numeric columns) were missing values may be present.

First a copy of the dataset is created, to not modify the original one. Then, we iterate over all the numerical features of the dataset. For each feature, we group the dataset by the `category` column. This means that all samples belonging to the same category will be consecutive. This is achieved by `modified_dataset.groupby(category_col)[feature]`. Then for each of these groups we find the mean of all non-missing values and at the same time we fill the missing values of that group with the mean. This is achieved by `transform(lambda x: x.fillna(x.mean()))`.

Before we use the function, we will define the names of the columns that contain numerical features and save them in the `cols` variable:

In [57]:
cols = dataset.select_dtypes(include='number').columns
cols

Index(['energy_kcal', 'fat_g', 'fatty_acids_sat_g', 'fatty_acids_monounsat_g',
       'fatty_acids_polyunsat_g', 'cholesterol_mg', 'carbohydrates_g',
       'sugars_g', 'starch_g', 'fibres_g', 'protein_g', 'salt_g', 'alcohol_g',
       'water_g', 'vit_A_activity_re_µg', 'vit_A_activity_rae_µg',
       'retinol_µg', 'beta_carotene_activity_µg', 'beta_carotene_µg',
       'vit_B1_mg', 'vit_B2_mg', 'vit_B6_mg', 'vit_B12_µg', 'niacin_mg',
       'folate_µg', 'panthotenic_acid_mg', 'vit_c_mg', 'vit_d_µg',
       'vit_e_activity_mg', 'potassium_mg', 'sodium_mg', 'chloride_mg',
       'calcium_mg', 'magnesium_mg', 'phosphorus_mg', 'iron_mg', 'iodide_µg',
       'zinc_mg'],
      dtype='object')

We pass the argument `'number'` to the `select_dtypes` method of Pandas dataframes to select the part of the dataset that contains only numeric values. After that, we select only the corresponding column names using the `columns` attribute.

Now imputation is performed:

In [58]:
dataset = impute_missing_values(dataset, category_col='category', feature_col_names=cols)

To confirm that the function has imputed the `NaN` values, let's look at the example from above:

In [59]:
dataset.loc[221, ['starch_g', 'category']]

starch_g   -0.524086
category        meat
Name: 221, dtype: object

Now it has the mean starch value for all meat products. To verify that there are no `NaN` values left, we can run this code snippet:

In [60]:
# create a boolean mask for NaN values
nan_mask = dataset.isna().any(axis=1)

# create a new DataFrame with samples containing NaN values
df_with_nan_only = dataset[nan_mask]

# check the dimensions
df_with_nan_only.shape

(0, 41)

As we can see, the resulting dataframe has zero rows. This means that there are no missing values left.

💡 **CHECKPOINT:**
- Why is data pre-processing important?
- Why do we need to remove samples/features with a lot of missing values?
- Why is imputation done per food category?

### 3.3 Train-Test Split <a id='train-test-split'></a>

Although in practice it happens after the real data preprocessing, sometimes it is considered to be part of the data pre-processing phase. As you can see from [Fig. 1](#ml-model), before the machine learning model is trained, the data is split in two parts: the **training data** and the **testing data**. Usually we use **70-80%** of the whole dataset for training and the remaining **20-30%** for testing. This is done to evaluate how well the model performs on an unseen dataset (the test set). If the model performs well on the test set, it has learned the data well and also manages to generalize to unseen data. If the model performance on the test set is poor, the model does not manage to generalize to unseen data and might have learned the training data too well (almost learned by heart).

Let us make an analogy to your courses at ETH. When you go and sit in an exam, you rarely or never find exactly the same questions that you have seen during the exercise sessions or classes. Also, you never know in advance which questions will be asked in the exam. The reason for all this constraints is that the professors want to evaluate you based on what you have understood from the course. If you already knew the questions of the exam you can easily prepare them beforehand, try to memorize the answers even if you understand nothing and then you get a really good grade from the course. But what happens next is that when you start working outside the doors of the university, you will probably not be able to solve the tasks at hand because of lack of understanding of certain concepts and set of skills. 

This is exactly what we want to avoid when training machine learning models. In the machine learning jargon, we want to have models that generalize well. This means that they can properly solve tasks which they have not encountered during training. And they can do perfectly so, if the training was successful and they have gained insights on data. In our analogy from above, if the students understand the concepts and can put them to practice, then they can be flexible in their jobs and solve tasks that they might not have encountered during the studies. All this happens because they will have the necessary set of skills from their training (education) years. 

Below you can see how we can split in practice a dataset. We will try two different methods for the train-test split. The first one is the simple train-test split using the `train_test_split()` function and the second one is the stratified train-test split using the again the same function but with an additional parameter, `stratify`. The difference between the two cases is that in the first case, the split of samples between train and test is random, while in the second case the proportions of categories in the train and test sets are preserved. This means that if 10% of the samples in the train set belong to the `meat` category, then approximately 10% of the test samples will belong to the `meat` category, as well.

We will start with the simple `train_test_split()` method from the `model_selection` module of the `sklearn` library, that we have imported above.

In [61]:
train_set_simple, test_set_simple = train_test_split(dataset, test_size=0.2, random_state=0)

We pass three arguments to the method. The first one is the dataset that will be split. The second one `test_size` determines the portion of the data that will be used for testing. Here we specify that `20%=0.2` will be used for testing, thus implicitly the remaining 80% for training. The method, depending on whether we pass the whole dataset or the features and the labels separately, will return 2 new dataframes in the first case (1x for the train set and 1x for the test set) or 4 new dataframes in the second case (1x for the train_features, 1x for the test features, 1x for the train labels and 1x for the test labels). The third argument, `random_state=0` makes sure that no matter how many times we execute the above code cell, it will produce the same split all the times. This means that the `train_set` and `test_set` will always have the same samples. If we omit that argument, then `train_set` and `test_set` will have different samples each time we execute the cell.

In order to make sure that the split is correct let us check the sizes of the new dataframes.

In [62]:
print("Shape of original dataset: ", dataset.shape)
print("Shape of train dataset: ", train_set_simple.shape)
print("Shape of test dataset: ", test_set_simple.shape)

Shape of original dataset:  (1084, 41)
Shape of train dataset:  (867, 41)
Shape of test dataset:  (217, 41)


This confirms our desired proportions for the split (80% train and 20% test). However, let us check the percentage of each class in the train and test datasets:

In [63]:
(train_set_simple['category'].value_counts() / train_set_simple.shape[0])*100

other                      19.838524
meat                       17.647059
cereals                    13.264129
vegetables                 11.995386
sweets                     11.188005
fruits                      9.573241
dairy                       9.457901
non_alcoholic_beverages     2.883506
alcoholic_beverages         2.076125
sauce                       1.268743
herbs                       0.692042
nuts                        0.115340
Name: category, dtype: float64

In [64]:
(test_set_simple['category'].value_counts() / test_set_simple.shape[0])*100

other                      25.806452
meat                       15.668203
cereals                    14.285714
vegetables                 10.599078
dairy                       9.677419
sweets                      9.677419
fruits                      7.373272
sauce                       2.304147
alcoholic_beverages         1.843318
non_alcoholic_beverages     1.382488
nuts                        0.921659
herbs                       0.460829
Name: category, dtype: float64

In both cases we used the `value_counts()` method to obtain the counts for each category. Then we divided these counts with the number of samples in the respective train and test sets by using the first entry of the `shape` attribute, which corresponds to the number of rows the set will have. Lastly, to convert the numbers to percentages we multiply them by 100. Thus we get the above values. As we can see the category percentages are not preserved. 

Now we will do the same procedure, but this time adding the `stratify` parameter to the `train_test_split` method. Here we set it to the `category` column of the dataset, since that will determine the proportions that should be preserved:

In [65]:
train_set_stratified, test_set_stratified = train_test_split(dataset, test_size=0.2, stratify=dataset['category'], random_state=0)

In [66]:
(train_set_stratified['category'].value_counts() / train_set_stratified.shape[0])*100

other                      20.991926
meat                       17.301038
cereals                    13.494810
vegetables                 11.764706
sweets                     10.841984
dairy                       9.457901
fruits                      9.111880
non_alcoholic_beverages     2.537486
alcoholic_beverages         2.076125
sauce                       1.499423
herbs                       0.692042
nuts                        0.230681
Name: category, dtype: float64

In [67]:
(test_set_stratified['category'].value_counts() / test_set_stratified.shape[0])*100

other                      21.198157
meat                       17.050691
cereals                    13.364055
vegetables                 11.520737
sweets                     11.059908
dairy                       9.677419
fruits                      9.216590
non_alcoholic_beverages     2.764977
alcoholic_beverages         1.843318
sauce                       1.382488
herbs                       0.460829
nuts                        0.460829
Name: category, dtype: float64

Here we see that the category percentages are more or less the same. From now on, we will use the stratified train-test split, since it will be useful in the next modules.

In [68]:
train_set = train_set_stratified.copy()
test_set = test_set_stratified.copy()

💡 **CHECKPOINT:**
- Why do we do the train-test split?

### 3.4 Standardization <a id='stand'></a>

Standardization is the process of transforming or rescaling the values of certain features so that they are comparable. This process is useful to the underlying steps of machine learning algorithms in finding the best model that can generalize the data. The formula for standardizing the values of a single feature column would be: $$ X^{(i)}_s = \frac{X^{(i)}-\mu}{\sigma} ,$$ where $X^{(i)}_s$ is the standardized value for sample `i`, $X^{(i)}$ is the original value of the feature for sample `i`, $\mu$ is the mean of all values of the feature (the whole column) and $\sigma$ is the standard deviation of all the values of that feature. We can depict this graphically as follows:

<center>
    <a id="standardization-illustration"></a>
    <img src="images/part1_intro/standardization.jpg" alt="Standardization" width="75%">
    <center><figcaption><em>Figure 3: Standardization in practice</em></figcaption></center>
</center>

As you can see, we first find the mean, column-wise for a feature. Then we find the standard deviation and then we apply the above formula for each of the samples to transform that feature into a standardized one. We repeat this procedure for all the feature columns one by one. In addition, standardization makes the distributions of the features more uniform.

**NOTE**: If we are in a supervised learning task, then we apply the scaling to all features but not to the label/target column. We avoid this because it might change the interpretation of the target variable. While in the case of unsupervised learning, the standardization may be applied to all columns since there is no target column.

Using this method, the mean of each corresponding feature is subtracted from the values and each of them is scaled to unit variance. Moreover, by using standardization the learning models will not be affected a lot by outliers and the learning process will be smoother. This happens due to some underlying properties of the standard normal distribution that are out of the scope of this notebook.

We can either code the procedure by ourselves or we can use ready-made code from the `sklearn` module. Here you will see an example of how to do standardization using `StandardScaler` from `sklearn`. We will continue using the dataset from the previous section. 
Again, we will start by importing the necessary modules:

In [69]:
standard_scaler = StandardScaler()
train_set_numerical_st = standard_scaler.fit_transform(train_set[cols])
test_set_numerical_st = standard_scaler.transform(test_set[cols])

After creating the `standard_scaler` object we use its method `fit_transform` **only in the numerical part of the `train_set`**. Then, **for test set, we use the `transform` method.** This is done to avoid any information leakage from the train to the test set, since it will be used to evaluate the machine learning models. In addition, since we assume that both the train and the test sets come from the same distribution, then there should be no difference between their means and standard deviations.  

Both these methods return *numpy arrays*. We will create two copies of the `train_set` and `test_set` dataframes and modify these copies to hold the standardized values instead of the original ones:

In [70]:
train_set_stand = train_set.copy()
train_set_stand[cols] = train_set_numerical_st
train_set_stand['split'] = 'train'

test_set_stand = test_set.copy()
test_set_stand[cols] = test_set_numerical_st
test_set_stand['split'] = 'test'

We also added a new column two both the train and test dataframes to indicate the split that the samples belong to.

Let us have a look how standardization changes the distribution of the features. For  this, we will plot the histogram of the values for the `fruit` category and the `energy_kcal` feature. Note that this step is not necessarily part of the machine learning pipeline we have seen so far. This is not needed for the model itself, but we will use it to give us an idea of what happens after standardization.

In [71]:
# from the whole train_set select only samples of the 'fruits' category
# for all samples of the 'fruit' category select only the `energy_kcal` column
original_fruit_energy = train_set[train_set['category']=='fruits']['energy_kcal']

# from the whole train_set_stand select only samples of the 'fruits' category
# for all samples of the 'fruit' category select only the `energy_kcal` column
standardized_fruit_energy = train_set_stand[train_set_stand['category']=='fruits']['energy_kcal']


We begin by selecting the `energy_kcal` features for all the samples in both the original and standardized train sets and see how the mean and standard deviation change after standardization:

In [72]:
train_set['energy_kcal'].describe()

count    8.670000e+02
mean     1.434198e-17
std      1.000577e+00
min     -1.197881e+00
25%     -7.616400e-01
50%     -3.081795e-01
75%      5.987417e-01
max      3.968126e+00
Name: energy_kcal, dtype: float64

In [73]:
train_set_stand['energy_kcal'].describe()

count    8.670000e+02
mean     1.229313e-17
std      1.000577e+00
min     -1.197881e+00
25%     -7.616400e-01
50%     -3.081795e-01
75%      5.987417e-01
max      3.968126e+00
Name: energy_kcal, dtype: float64

We use the `describe()` method of the pandas dataframe to get some statistics about the distribution of the values in the `energy_kcal` column.

By analyzing these statistics, we can see that after standardization the data has become more uniform. This is evident when we see the mean and standard deviation before and after standardization. The mean before standardization was $206.24$, while the mean after standardization is $-7.38 \times 10^{-17}$. The standard deviation has also changed. Before the standardization it was $169.21$ while after the standardization it is $1.0$, so the range of values is smaller and therefore the distribution of values more uniform.

Now we are going to concatenate the separate `train_set_stand` and `test_set_stand` dataframes in a single standardized dataset and save it in a file:

In [74]:
# concatenate the standardized train and test dataframes
# sort them using the ID/index column
dataset_stand = pd.concat([train_set_stand, test_set_stand]).sort_index()
dataset_stand

# save the standardized dataframe
dataset_stand.to_csv('data/swiss_food_composition_proc.csv')

Having processed our food composition dataset, we can now apply supervised and unsupervised ML methods to it in the next two modules.

💡 **CHECKPOINT:**
- Why is standardization important?
- Why `fit_transform()` is applied on the train set and `transform()` on the test set?

**References:**
 - "Machine Learning with Pytorch and Scikit-Learn" - Sebastian Raschka, Yuxi Liu, Vahid Mirjalili, Dmytro Dzhulgakov.
 - Raw data from https://naehrwertdaten.ch/en/downloads/.